# Zip a bunch of spm outputs

In [1]:
from bids.layout import BIDSLayout
from nipype.interfaces import afni 
from nipype.interfaces.io import BIDSDataGrabber, DataFinder, DataSink, DataGrabber
import nipype.pipeline as pe
import nipype as ni
from nipype.interfaces.utility import Function
import nipype.interfaces.fsl.maths as fsl
from nipype.interfaces import spm as spm
from nipype.algorithms.misc import Gunzip 
import pandas as pd
import os, re, json
# https://nipype.readthedocs.io/en/0.11.0/users/spmmcr.html

matlab_cmd = '/opt/spm12-r7219/run_spm12.sh /opt/matlabmcr-2010a/v713/ script'
spm.SPMCommand.set_mlab_paths(matlab_cmd=matlab_cmd, use_mcr=True)

201130-09:20:26,172 nipype.utils WARNING:
	 A newer version (1.5.1) of nipy/nipype is available. You are using 1.5.0


In [2]:
Basedir = "/scratch/qbi/uqkgarn1/STRIWP1/"
zppn = pe.Workflow(name="zppn") # workflow to run the analysis

# data grabbers and sinkers

Edit the below to get the files you would like to zip

In [3]:
# THIS CODE DEFINITELY WORKS
dg = pe.Node(DataGrabber(infields=['sub', 'TR'], 
                        outfields=['ts']), name='grabber')
dg.inputs.base_dir = "/scratch/qbi/uqkgarn1/STRIWP1/"
dg.inputs.sort_filelist = True
dg.inputs.template='*'
dg.inputs.template_args = {'ts': [['sub', 'TR']]}
dg.inputs.field_template = {'ts': '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-%s/TR%s/FLGLM/spmT_*.nii'}

In [4]:
dg.inputs.sub = '03'
dg.inputs.TR = '1920'
res = dg.run()
res.outputs

201130-09:20:28,499 nipype.workflow INFO:
	 [Node] Setting-up "grabber" in "/tmp/tmpcx64xwx9/grabber".
201130-09:20:28,504 nipype.workflow INFO:
	 [Node] Running "grabber" ("nipype.interfaces.io.DataGrabber")
201130-09:20:28,510 nipype.workflow INFO:
	 [Node] Finished "grabber".



ts = ['/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1920/FLGLM/spmT_0001.nii', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1920/FLGLM/spmT_0002.nii', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1920/FLGLM/spmT_0003.nii', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1920/FLGLM/spmT_0005.nii', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1920/FLGLM/spmT_0006.nii', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1920/FLGLM/spmT_0007.nii', '/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1920/FLGLM/spmT_0009.nii']

In [5]:
datinf = pe.Node(ni.IdentityInterface(fields=['sub', 'TR']),
                 name='dat-info')
datinf.iterables = [('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]
#datinf.iterables = [('sub', ['03']), ('TR', ['1920'])]
datinf.iterables 

[('sub', ['01', '02', '03', '04', '05']), ('TR', ['700', '1510', '1920'])]

In [6]:
def printSubPath(fullFilePath):
    # given the resms output of the datagrabber, recreate the required filepath for saving the data
    import os
    import re
    fname = os.path.normpath(fullFilePath[0])
    l = fname.split(os.sep)
    name = [s for s in l if re.search('sub', s)][0], [t for t in l if re.search('TR([0-9])', t)][0]
    name = '/'.join(name)
    return name

In [7]:
#printSubPath(res.outputs.ts)

# Data sink

In [8]:
ds = pe.Node(DataSink(), name='sink-stuff')
ds.inputs.base_directory = "/scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/"
substitutions = [('_TR_([0-9]*)_sub_([0-9]*)', ''),
                 ('_ratio([0-9]*)', ''),
                 ('_sqrtsqrs([0-9]*)', '')]
ds.inputs.regexp_substitutions = substitutions

# Zip files

In [9]:
from nipype.interfaces.base import (
    TraitedSpec,
    CommandLineInputSpec,
    CommandLine,
    File
)
import os

class GZipInputSpec(CommandLineInputSpec):
    in_file = File(desc="File", exists=True, mandatory=True, argstr="%s")

class GZipOutputSpec(TraitedSpec):
    out_file = File(desc = "Zip file", exists = True)

class GZipTask(CommandLine):
    input_spec = GZipInputSpec
    output_spec = GZipOutputSpec
    _cmd = 'gzip'

    def _list_outputs(self):
            outputs = self.output_spec().get()
            outputs['out_file'] = os.path.abspath(self.inputs.in_file + ".gz")
            return outputs

In [10]:
gzip = pe.MapNode(GZipTask(), iterfield='in_file', name='gzip')

In [11]:
zppn.connect([ (datinf, dg, [('sub', 'sub')]),
               (datinf, dg, [('TR', 'TR')]),
               (dg, ds, [(('ts', printSubPath), 'container')]),
               (dg, gzip, [('ts', 'in_file')]),
               (gzip, ds, [('out_file', 'FLGLM.@gzip')])
             ])

In [12]:
zppn.run()

201130-09:20:28,583 nipype.workflow INFO:
	 Workflow zppn settings: ['check', 'execution', 'logging', 'monitoring']
201130-09:20:28,636 nipype.workflow INFO:
	 Running serially.
201130-09:20:28,638 nipype.workflow INFO:
	 [Node] Setting-up "zppn.grabber" in "/tmp/tmpcx64xwx9/grabber".
201130-09:20:28,642 nipype.workflow INFO:
	 [Node] Running "grabber" ("nipype.interfaces.io.DataGrabber")
201130-09:20:28,648 nipype.workflow INFO:
	 [Node] Finished "zppn.grabber".
201130-09:20:28,649 nipype.workflow INFO:
	 [Node] Setting-up "zppn.gzip" in "/tmp/tmpxqmj02_h/zppn/_TR_1920_sub_05/gzip".
201130-09:20:28,655 nipype.workflow INFO:
	 [Node] Setting-up "_gzip0" in "/tmp/tmpxqmj02_h/zppn/_TR_1920_sub_05/gzip/mapflow/_gzip0".
201130-09:20:28,658 nipype.workflow INFO:
	 [Node] Running "_gzip0" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR1920/FLGLM/spmT_0001.nii
201130-09:20:28,922 nipype.workflow INFO:
	 [Node] Fin

201130-09:20:32,196 nipype.workflow INFO:
	 [Node] Running "_gzip0" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/FLGLM/spmT_0001.nii
201130-09:20:32,376 nipype.workflow INFO:
	 [Node] Finished "_gzip0".
201130-09:20:32,378 nipype.workflow INFO:
	 [Node] Setting-up "_gzip1" in "/tmp/tmp1buzmpgg/zppn/_TR_700_sub_05/gzip/mapflow/_gzip1".
201130-09:20:32,381 nipype.workflow INFO:
	 [Node] Running "_gzip1" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-05/TR700/FLGLM/spmT_0002.nii
201130-09:20:32,540 nipype.workflow INFO:
	 [Node] Finished "_gzip1".
201130-09:20:32,542 nipype.workflow INFO:
	 [Node] Setting-up "_gzip2" in "/tmp/tmp1buzmpgg/zppn/_TR_700_sub_05/gzip/mapflow/_gzip2".
201130-09:20:32,547 nipype.workflow INFO:
	 [Node] Running "_gzip2" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn

201130-09:20:35,951 nipype.workflow INFO:
	 [Node] Finished "_gzip1".
201130-09:20:35,954 nipype.workflow INFO:
	 [Node] Setting-up "_gzip2" in "/tmp/tmpk2aabe7v/zppn/_TR_1510_sub_04/gzip/mapflow/_gzip2".
201130-09:20:35,957 nipype.workflow INFO:
	 [Node] Running "_gzip2" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-04/TR1510/FLGLM/spmT_0003.nii
201130-09:20:36,191 nipype.workflow INFO:
	 [Node] Finished "_gzip2".
201130-09:20:36,193 nipype.workflow INFO:
	 [Node] Setting-up "_gzip3" in "/tmp/tmpk2aabe7v/zppn/_TR_1510_sub_04/gzip/mapflow/_gzip3".
201130-09:20:36,197 nipype.workflow INFO:
	 [Node] Running "_gzip3" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-04/TR1510/FLGLM/spmT_0005.nii
201130-09:20:36,456 nipype.workflow INFO:
	 [Node] Finished "_gzip3".
201130-09:20:36,458 nipype.workflow INFO:
	 [Node] Setting-up "_gzip4" in "/tmp/

201130-09:20:39,730 nipype.workflow INFO:
	 [Node] Finished "_gzip3".
201130-09:20:39,732 nipype.workflow INFO:
	 [Node] Setting-up "_gzip4" in "/tmp/tmpfknzeu4f/zppn/_TR_1920_sub_03/gzip/mapflow/_gzip4".
201130-09:20:39,736 nipype.workflow INFO:
	 [Node] Running "_gzip4" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1920/FLGLM/spmT_0006.nii
201130-09:20:39,937 nipype.workflow INFO:
	 [Node] Finished "_gzip4".
201130-09:20:39,939 nipype.workflow INFO:
	 [Node] Setting-up "_gzip5" in "/tmp/tmpfknzeu4f/zppn/_TR_1920_sub_03/gzip/mapflow/_gzip5".
201130-09:20:39,943 nipype.workflow INFO:
	 [Node] Running "_gzip5" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR1920/FLGLM/spmT_0007.nii
201130-09:20:40,141 nipype.workflow INFO:
	 [Node] Finished "_gzip5".
201130-09:20:40,143 nipype.workflow INFO:
	 [Node] Setting-up "_gzip6" in "/tmp/

201130-09:20:43,178 nipype.workflow INFO:
	 [Node] Finished "_gzip5".
201130-09:20:43,179 nipype.workflow INFO:
	 [Node] Setting-up "_gzip6" in "/tmp/tmp2r5dkqtp/zppn/_TR_700_sub_03/gzip/mapflow/_gzip6".
201130-09:20:43,183 nipype.workflow INFO:
	 [Node] Running "_gzip6" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-03/TR700/FLGLM/spmT_0009.nii
201130-09:20:43,395 nipype.workflow INFO:
	 [Node] Finished "_gzip6".
201130-09:20:43,401 nipype.workflow INFO:
	 [Node] Finished "zppn.gzip".
201130-09:20:43,402 nipype.workflow INFO:
	 [Node] Setting-up "zppn.sink-stuff" in "/tmp/tmprhfe9wv1/zppn/_TR_700_sub_03/sink-stuff".
201130-09:20:43,410 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
201130-09:20:43,417 nipype.workflow INFO:
	 [Node] Finished "zppn.sink-stuff".
201130-09:20:43,418 nipype.workflow INFO:
	 [Node] Setting-up "zppn.grabber" in "/tmp/tmpcx64xwx9/grabber".
201130-

201130-09:20:47,133 nipype.workflow INFO:
	 [Node] Running "sink-stuff" ("nipype.interfaces.io.DataSink")
201130-09:20:47,138 nipype.workflow INFO:
	 [Node] Finished "zppn.sink-stuff".
201130-09:20:47,139 nipype.workflow INFO:
	 [Node] Setting-up "zppn.grabber" in "/tmp/tmpcx64xwx9/grabber".
201130-09:20:47,143 nipype.workflow INFO:
	 [Node] Running "grabber" ("nipype.interfaces.io.DataGrabber")
201130-09:20:47,148 nipype.workflow INFO:
	 [Node] Finished "zppn.grabber".
201130-09:20:47,149 nipype.workflow INFO:
	 [Node] Setting-up "zppn.gzip" in "/tmp/tmppgy8ai4t/zppn/_TR_700_sub_02/gzip".
201130-09:20:47,155 nipype.workflow INFO:
	 [Node] Setting-up "_gzip0" in "/tmp/tmppgy8ai4t/zppn/_TR_700_sub_02/gzip/mapflow/_gzip0".
201130-09:20:47,158 nipype.workflow INFO:
	 [Node] Running "_gzip0" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-02/TR700/FLGLM/spmT_0001.nii
201130-09:20:47,307 nipype.workflow INFO:
	 [Node]

201130-09:20:50,415 nipype.workflow INFO:
	 [Node] Running "_gzip0" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR1510/FLGLM/spmT_0001.nii
201130-09:20:50,623 nipype.workflow INFO:
	 [Node] Finished "_gzip0".
201130-09:20:50,625 nipype.workflow INFO:
	 [Node] Setting-up "_gzip1" in "/tmp/tmp1byxnpss/zppn/_TR_1510_sub_01/gzip/mapflow/_gzip1".
201130-09:20:50,629 nipype.workflow INFO:
	 [Node] Running "_gzip1" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqkgarn1/STRIWP1/derivatives/glmTDFAST/sub-01/TR1510/FLGLM/spmT_0002.nii
201130-09:20:50,804 nipype.workflow INFO:
	 [Node] Finished "_gzip1".
201130-09:20:50,806 nipype.workflow INFO:
	 [Node] Setting-up "_gzip2" in "/tmp/tmp1byxnpss/zppn/_TR_1510_sub_01/gzip/mapflow/_gzip2".
201130-09:20:50,810 nipype.workflow INFO:
	 [Node] Running "_gzip2" ("__main__.GZipTask"), a CommandLine Interface with command:
gzip /scratch/qbi/uqk